In [17]:
import sys
sys.setrecursionlimit(10**6)

with open("23input.txt") as file:
    forest = [list(line) for line in file.read().splitlines()]

# Part 1
def is_valid(next, current, forest):
    rows, cols = len(forest), len(forest[0])
    i, j = next[0], next[1]
    if 0 <= i < rows and 0 <= j < cols:
        if forest[i][j] != "#":
            if forest[i][j] == ">":
                if current[0] - i == 0 and current[1] - j == -1:
                    return True
            elif forest[i][j] == "<":
                if current[0] - i == 0 and current[1] - j == 1:
                    return True
            elif forest[i][j] == "^":
                if current[0] - i == 1 and current[1] - j == 0:
                    return True
            elif forest[i][j] == "v":
                if current[0] - i == -1 and current[1] - j == 0:
                    return True
            elif forest[i][j] == ".":
                return True

    return False

def paths(start, end, forest, visited, num_steps, all_paths, memory):
    visited.append(start)
    i, j = start[0], start[1]
    neighbors = [(i+1, j), (i-1, j), (i, j+1), (i, j-1)]
    if (i, j) == end:
        all_paths.append(num_steps)
    elif (i, j) == (0, 1):
        neighbors = [(i+1, j), (i, j+1)]
    for n in neighbors:
        if n not in visited:
            if is_valid(n, start, forest):
                if n not in memory or num_steps + 1 > memory[n]:
                    paths(n, end, forest, visited, num_steps + 1, all_paths, memory)
                    memory[n] = num_steps + 1
    visited.remove(start)

def find_longest_hike(start, end, forest):
    all_paths = []
    memory = {}
    paths(start, end, forest, [], 0, all_paths,memory)
    return max(all_paths, default=0)

longest_hike_part_1 = find_longest_hike((0, 1), (len(forest)-1, len(forest[0])-2), forest)
print(longest_hike_part_1)

# Part 2 - a different approach than in part 1, because using part 1 would be too computationally demanding (trust me, I tried...), therefore I tried "contracting
# the graph (when there is a long hall, replace it with one edge of higher weight)
nodes_coordinates, label, edges = set(), 0, {}

for i, row in enumerate(forest):
    for j, tile in enumerate(row):
        if tile != "#":
            nodes_coordinates.add((i, j))
            forest[i][j], label = label, label + 1

for i, row in enumerate(forest):
    for j, tile in enumerate(row):
        if tile != "#":
            neighbors = [(i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)]
            edges.setdefault(forest[i][j], {}).update({forest[n[0]][n[1]]: 1 for n in neighbors if n in nodes_coordinates})

def compress(nodes_coordinates, edges):
    for tile in nodes_coordinates:
        a, node = forest[tile[0]][tile[1]], edges[forest[tile[0]][tile[1]]]
        if len(node) == 2:
            nodes_coordinates.remove(tile)
            b, c = list(node.keys())
            edges[b][c], edges[c][b] = sum(edges[a].values()), sum(edges[a].values())
            del edges[b][a], edges[c][a], edges[a]
            return compress(nodes_coordinates, edges)
    return edges

nodes_coordinates = compress(nodes_coordinates, edges)

def paths_2(start, end, nodes_coordinates, visited, num_steps, all_paths): 
    visited.append(start)
    neighbors = list(nodes_coordinates[start].keys())
    if start == end:
        all_paths.append(num_steps)
    for n in neighbors:
        if n not in visited:
            paths_2(n, end, nodes_coordinates, visited, num_steps + nodes_coordinates[start][n], all_paths)
    visited.remove(start) 

def find_longest_hike_2(start, end, nodes_coordinates):
    all_paths = []
    paths_2(start, end, nodes_coordinates, [], 0, all_paths)
    return max(all_paths, default = 0)


find_longest_hike_2(0, label - 1, nodes_coordinates)

2354


6686